# **Prototying SQLite Database**

I want the API to be capable of running a pre-processing pipeline on a user's uploaded `conversations.json`. That pre-processing pipeline will populate a SQLite database w/ data that can then be used by the FastAPI!

Within this notebook, I'm going to prototype that database's initialization. This code will eventually be moved into a singular pipeline that can be called by the FastAPI.


# Setup

The cells below will set up the rest of the notebook.

I'll start by configuring the kernel:


In [ ]:
# Change the working directory 
%cd ..

# Enable the autoreload extension, which will automatically load in new code as it's written
%load_ext autoreload
%autoreload 2

Now I'll import some necessary modules:


In [ ]:
# General imports
import math
import json
import sqlite3

# Third-party imports
import pandas as pd
from tqdm import tqdm
import numpy as np
from umap import UMAP

# Project imports
import utils.data_parsing as data_utils
import utils.llm_enrichment as llm_utils
import utils.openai as openai_utils
import utils.clusters as cluster_utils
import utils.pipelines as pipeline_utils

# Parameterizing Pipeline

Below, we're going to parameterize the overall pipeline a bit:


In [ ]:
# Indicate where the conversations.json file is located
conversations_json_file_path = "data/export/conversations.json"

# Load in the conversations.json file
with open(conversations_json_file_path) as f:
    conversations = json.load(f)

# Declare a constant indicating the number of characters we'll include in the context
max_n_chars_in_conversation_context = 4_000

# TODO: I need to remove this eventually, but: I'm subsetting the conversations for testing purposes
conversations = conversations[:200]

# Initializing Tables

First, we'll need to initialize a couple of different tables. Before doing that, I'll create a database:


In [ ]:
# Create a connection to the database
db_conn = sqlite3.connect("data/test_db.db")

# Create a cursor object
db_cursor = db_conn.cursor()

# Drop the conversations + clusters tables if they exist
db_cursor.execute("DROP TABLE IF EXISTS conversations")
db_cursor.execute("DROP TABLE IF EXISTS clusters")

### **`conversations`**

This table will store information about each ChatGPT conversation!


In [ ]:
# Define the query that'll generate the conversations table
create_conversations_table_query = """
CREATE TABLE conversations (
    conversation_id TEXT PRIMARY_KEY, -- This is the unique identifier for the conversation
    title TEXT NOT NULL, -- This is the title of the conversation
    create_time INTEGER, -- This is a Unix timestamp indicating when the conversation was created
    default_model_slug TEXT, -- This is the model that was used to generate the conversation
    raw_messages_data TEXT, -- This is a JSON string containing the entire conversation's raw messages data
    messages_markdown TEXT, -- This is a Markdown string containing the entire conversation
    summary TEXT, -- This is a summary of the conversation
    tags TEXT, -- This is a JSON string containing a list of tags associated with the conversation
    embedding BLOB, -- This is a binary large object containing the conversation's embedding
    umap_x REAL, -- This is the UMAP x-coordinate of the conversation's embedding
    umap_y REAL -- This is the UMAP y-coordinate of the conversation's embedding
)
"""

# Execute the query
db_cursor.execute(create_conversations_table_query)

### **`clusters`**

Next up: we're going to create a table called `clusters`. Whenever the user creates some semantic clusters for their data, it'll be stored here.


In [ ]:
# Define the query that'll generate the clusters table
create_clusters_table_query = """
CREATE TABLE clusters (
    cluster_solution_id TEXT,
    cluster_id TEXT,
    conversation_ids TEXT, -- This is a JSON string containing a list of the conversation IDs that belong to this cluster
    centroid_conversation_ids TEXT, -- This is a JSON string containing a list of the conversation IDs that are the centroids of this cluster
    centroid_embedding BLOB, -- This is a binary large object containing the centroid embedding of the cluster
    cluster_size INTEGER, -- The number of conversations in the cluster
    cluster_label TEXT, -- The label of the cluster
    cluster_description TEXT, -- A description of the cluster
    tag_counts TEXT, -- This is a JSON string containing the counts of particular tags in the cluster
    mean_cosine_similarity REAL, -- The mean cosine similarity between points and centroid
    cluster_radius REAL, -- The maximum distance from any point to centroid
    silhouette_score REAL, -- The silhouette score for this cluster
    centroid_umap_x REAL, -- The UMAP x-coordinate of the centroid
    centroid_umap_y REAL, -- The UMAP y-coordinate of the centroid
    PRIMARY KEY (cluster_solution_id, cluster_id)
)
"""

# Execute the query
db_cursor.execute(create_clusters_table_query)

# Populating Tables

Next up: I'm going to populate the tables a bit!


### Adding Initial Data to **`conversations`**

Below, I'll populate the `conversations` table with data from the `conversations.json`. I'll start by extracting the necessary data:


In [ ]:
# Collect data to insert into the conversations table
conversations_data_to_insert = []
for conversation in tqdm(
    iterable=conversations,
    desc="Processing conversations",
):

    # Extract a DataFrame representation of the messages in this conversation
    messasges_df = data_utils.extract_longest_conversation_df(
        mapping=conversation.get("mapping", {})
    )

    # Generate the data to insert
    cur_conversation_data = {
        "conversation_id": conversation.get("conversation_id"),
        "title": conversation.get("title", "UNTITLED CONVERSATION"),
        "create_time": conversation.get("create_time"),
        "default_model_slug": conversation.get("default_model_slug"),
        "raw_messages_data": messasges_df.to_json(),
        "messages_markdown": data_utils.extract_simple_conversation_markdown(
            conversation_df=messasges_df
        ),
    }

    # Add the data to the list
    conversations_data_to_insert.append(cur_conversation_data)

Next, I'll add all of this data to the SQLite database:


In [ ]:
# Insert the data into the conversations table
insert_conversations_query = """
INSERT INTO conversations (
    conversation_id,
    title,
    create_time,
    default_model_slug,
    raw_messages_data,
    messages_markdown
) VALUES (
    :conversation_id,
    :title,
    :create_time,
    :default_model_slug,
    :raw_messages_data,
    :messages_markdown
)
"""

# Execute the query
db_cursor.executemany(insert_conversations_query, conversations_data_to_insert)

# Commit the changes
db_conn.commit()

### Enriching Data in `conversations` Table

Next up, we'll want to do a little "data enrichment" for the conversations table. During this step, we'll add LLM-generated summaries & tags, as well as embeddings.

I'll start by running the LLM enrichment:


In [ ]:
# Enrich the conversations with LLM summaries & tags
llm_enriched_conversations_df = llm_utils.enrich_conversations_with_summaries_and_tags(
    conversations_df=pd.DataFrame(conversations_data_to_insert),
    max_chars_per_conversation_context=max_n_chars_in_conversation_context,
)

Next, I'll add the `summary` and `tags` to the database:


In [ ]:
# Collect data to insert into the conversations table
conversations_data_to_insert = [
    {
        "conversation_id": row.conversation_id,
        "summary": row.summary,
        "tags": json.dumps(row.tags),
    }
    for row in llm_enriched_conversations_df.itertuples()
]

# Insert the data into the conversations table
insert_conversations_query = """
UPDATE conversations
SET
    summary = :summary,
    tags = :tags
WHERE
    conversation_id = :conversation_id
"""

# Execute the query
db_cursor.executemany(insert_conversations_query, conversations_data_to_insert)

# Commit the changes
db_conn.commit()

Next up: I'm going to embed the conversations.


In [ ]:
# Generate the embeddings for the conversations
embs = openai_utils.generate_embeddings_for_texts(
    text_list=[
        f"{row.title}\nTags: {', '.join(row.tags)}\nSummary: {row.summary}\nConversation: {row.messages_markdown[:max_n_chars_in_conversation_context]}"
        for row in llm_enriched_conversations_df.itertuples()
    ]
)

# Create a copy of the enriched conversations DataFrame
conversation_emb_df = llm_enriched_conversations_df.copy()[["conversation_id"]]

# Add the embeddings to the DataFrame
conversation_emb_df["embedding"] = embs.tolist()

Now, I'll generate a UMAP model for the embeddings, and use it to generate UMAP projections of the embeddings:


In [ ]:
# TODO: I should figure out how to optimize this to go faster
# Initialize and fit a UMAP model on the embeddings
umap_model = UMAP(n_components=2)

# Add the UMAP coordinates to the DataFrame
conversation_emb_df[["umap_x", "umap_y"]] = umap_model.fit_transform(
    np.stack(conversation_emb_df.embedding)
)

With the conversations embedded, I can add them to the database:


In [ ]:
# Convert list of floats to BLOB before storing
conversations_data_to_insert = [
    {
        "conversation_id": row.conversation_id,
        "embedding": np.array(
            row.embedding, dtype=np.float32
        ).tobytes(),  # Convert the list of floats to a BLOB
        "umap_x": row.umap_x,
        "umap_y": row.umap_y,
    }
    for row in conversation_emb_df.itertuples()
]

# Update the conversations table
insert_conversations_query = """
UPDATE conversations
SET
    embedding = :embedding,
    umap_x = :umap_x,
    umap_y = :umap_y
WHERE
    conversation_id = :conversation_id
"""

# Execute the query
db_cursor.executemany(insert_conversations_query, conversations_data_to_insert)

# Commit the changes
db_conn.commit()

### Populating the **`clusters`** Table

The application will allow users to create their own clustering solutions; still, though, to get them started, we'll create a solution based on the number of conversations they've got.

First, we'll cluster the conversations and determine some metrics about those clusters:

In [ ]:
# TODO: Figure out a better default for determining the number of clusters
n_clusters = min(math.ceil(math.sqrt(len(conversation_emb_df))), 24)

# Cluster the conversations
clusters_df = cluster_utils.cluster_conversations(
    conversation_embs_df=conversation_emb_df.merge(
        llm_enriched_conversations_df[["conversation_id", "tags"]],
        on="conversation_id",
    ),
    n_clusters=n_clusters,
)

# Calculate some cluster metrics
cluster_metrics_df = cluster_utils.calculate_cluster_metrics(clusters_df)

# Add UMAP coordinates to the cluster_metrics_df
cluster_metrics_df[["umap_x", "umap_y"]] = umap_model.fit_transform(
    np.stack(cluster_metrics_df.embedding_centroid)
)

Next up: I'm going to use LLMs to label these clusters.

In [ ]:
# Run the LLM labeling for each of the conversation clusters
labeled_clusters_df = llm_utils.label_conversation_clusters(
    conversations_df=llm_enriched_conversations_df,
    cluster_metrics_df=cluster_metrics_df,
)

Finally, I'll populate the `clusters` table:

In [ ]:
# Convert the cluster data into a format suitable for SQLite insertion
clusters_data_to_insert = []
for row in labeled_clusters_df.itertuples():
    clusters_data_to_insert.append(
        {
            "cluster_solution_id": f"kmeans_{n_clusters}",
            "cluster_id": row.cluster,
            "conversation_ids": json.dumps(row.all_conversation_ids),
            "centroid_conversation_ids": json.dumps(row.centroid_conversation_ids),
            "centroid_embedding": np.array(
                row.embedding_centroid, dtype=np.float32
            ).tobytes(),
            "cluster_size": row.n_conversations,
            "cluster_label": row.cluster_label,
            "cluster_description": row.cluster_description,
            "tag_counts": json.dumps(row.tag_counts),
            "mean_cosine_similarity": row.mean_cosine_similarity,
            "cluster_radius": row.cluster_radius,
            "silhouette_score": row.silhouette_score,
            "centroid_umap_x": row.umap_x,
            "centroid_umap_y": row.umap_y,
        }
    )

# Insert the data into the clusters table
insert_clusters_query = """
INSERT INTO clusters (
    cluster_solution_id,
    cluster_id,
    conversation_ids,
    centroid_conversation_ids,
    centroid_embedding,
    cluster_size,
    cluster_label,
    cluster_description,
    tag_counts,
    mean_cosine_similarity,
    cluster_radius,
    silhouette_score,
    centroid_umap_x,
    centroid_umap_y
) VALUES (
    :cluster_solution_id,
    :cluster_id,
    :conversation_ids,
    :centroid_conversation_ids,
    :centroid_embedding,
    :cluster_size,
    :cluster_label,
    :cluster_description,
    :tag_counts,
    :mean_cosine_similarity,
    :cluster_radius,
    :silhouette_score,
    :centroid_umap_x,
    :centroid_umap_y
)
"""

# Execute the query
db_cursor.executemany(insert_clusters_query, clusters_data_to_insert)

# Commit the changes
db_conn.commit()

# Functionalizing Pipeline
I've combined all of the above code into a single method, which I'll test below:

In [ ]:
# Indicate where the conversations.json file is located
conversations_json_file_path = "data/export/conversations.json"

# Load in the conversations.json file
with open(conversations_json_file_path) as f:
    conversations = json.load(f)

pipeline_utils.preprocess_conversation_data(
    conversations=conversations,
    db_path="data/parsed_export_data.db",
)